In [1]:
from pymystem3 import Mystem
from os import listdir

In [2]:
import pymorphy2
import pymorphy2_dicts_ru

In [3]:
pymorphy = pymorphy2.MorphAnalyzer()

In [4]:
def concat(lists):
    result = []
    for lst in lists:
        for elem in lst:
            result.append(elem)
    return result

In [5]:
# удаление из списка лемм "мусора", не имеющего характеристик:
# переносов строк, знаков препинания, иностранных слов и др.
def delete_trash_lemmas(words):
    return list(filter(lambda l: l.get('analysis') != None and len(l.get('analysis')) > 0, words))

In [56]:
def getPos(word):
    params = pymorphy.parse(word)[0]
    return params.tag.POS

In [57]:
# морфологический анализатор
mystem = Mystem()

In [58]:
# директория хранения текстовых файлов для тестирования
text_path = 'texts'

# считывание текстовых файлов из заданной директории в список текстов
texts = [ open(text_path + '/' + filename, 'r').read() for filename in listdir(text_path) ]

In [59]:
# предикат: является ли слово разделителем
def isSeparator(word):
    return word.get('analysis') == None

# получить текст слова
def getText(word):
    return word['text']

# извлечение из текста множества всех разделителей
separators = set(concat([ list(map(getText, filter(isSeparator, mystem.analyze(text)))) for text in texts ]))

In [60]:
# посмотрим на них
separators

{'\n',
 ' ',
 ' (',
 ' «',
 ' ‒ ',
 ' –',
 ' – ',
 ' — ',
 ' — «',
 '!',
 ') ',
 '), ',
 '); ',
 ',',
 ', ',
 ', ‒ ',
 ', – ',
 ', — ',
 '-',
 '- ',
 '.',
 '. ',
 '/',
 '1917',
 '1941',
 ':\n',
 ': ',
 ': «',
 '; ',
 '?',
 '? ',
 '«',
 '»',
 '» ',
 '» (',
 '» ‒ ',
 '», ',
 '», – ',
 '– ',
 '…',
 '… '}

In [61]:
# индексы элементов, ошибочно выбранных разделителями
excludeSeparators = set(['1917', '- ', '1941', ' ', '-'])

# разделители, завершающие простые предложения
endSeparators = separators - excludeSeparators
endSeparators

{'\n',
 ' (',
 ' «',
 ' ‒ ',
 ' –',
 ' – ',
 ' — ',
 ' — «',
 '!',
 ') ',
 '), ',
 '); ',
 ',',
 ', ',
 ', ‒ ',
 ', – ',
 ', — ',
 '.',
 '. ',
 '/',
 ':\n',
 ': ',
 ': «',
 '; ',
 '?',
 '? ',
 '«',
 '»',
 '» ',
 '» (',
 '» ‒ ',
 '», ',
 '», – ',
 '– ',
 '…',
 '… '}

In [62]:
def print_group(group):
    if len(group) > 1:
        print('{', end='')

        for word in group:
            print(word, end='')

        print('}', end='')
    else:
        for word in group:
            print(word, end='')

def print_sentence(sentence):
    if isinstance(sentence, list):
        print('[', end='')

        for group in sentence:
            print_group(group)

        print(']', end='')
    else:
        print(sentence, end='')

def print_analyzed_text(text):
    for sentence in text:
        print_sentence(sentence)

In [71]:
for text in texts:
    # извлечение из текста списка слова со всеми характеристиками с помощью морфологического анализатора
    words = list(map(getText, mystem.analyze(text)))
    
    # список всех сегментов-предложений, полученных из обрабатываемого текста
    analyzed_text = []
    # список сегментов в рамках предложения
    sentence = []
    # список слов неразрывной синтаксической группы
    group = []
    
    i = 0
    while i < len(words):
        word = words[i]
        if word in endSeparators:
            if group != []:
                sentence.append(group)
                group = []

            if sentence != []:
                analyzed_text.append(sentence)
                sentence = []

            analyzed_text.append(word)
        elif word in separators and group == []:
            sentence.append([word])
        elif i + 2 < len(words) and words[i+1] not in endSeparators:
            if getPos(words[i]) == 'PREP' and getPos(words[i+2]) == 'NOUN':
                group = [ word, words[i+1], words[i+2] ]
                i += 2
                sentence.append(group)
                group = []
            elif getPos(words[i]) == 'ADJF' and getPos(words[i+2]) == 'NOUN':
                group = [ word, words[i+1], words[i+2] ]
                i += 2
                sentence.append(group)
                group = []
            else:
                sentence.append([word])
        else:
            sentence.append([word])
        i += 1
    else:
        if group != []:
            sentence.append(group)
        
        if sentence != []:
            analyzed_text.append(sentence)
    
    print_analyzed_text(analyzed_text)

[{Профессорская дача} {на берегу} {Финского залива}]. [{В отсутствие} хозяина], [друга {моего отца}], [{нашей семье} позволялось там жить]. [Даже {спустя десятилетия} помню], [как после {утомительной дороги} {из города} меня обволакивала прохлада {деревянного дома}], [как собирала растрясшееся], [распавшееся {в экипаже} тело]. [{Эта прохлада} не была связана {со свежестью}], [скорее], [как ни странно], — [с {упоительной затхлостью}], [в которой слились ароматы старых книг и многочисленных {океанских трофеев}], [непонятно как доставшихся профессору-юристу]. [Распространяя {солоноватый запах}], [{на полках} лежали засушенные {морские звёзды}], [{перламутровые раковины}], [{резные маски}], [{пробковый шлем} и даже игла рыбы-иглы].

[Аккуратно отодвигая дары моря], [я доставал {с полок} книги], [садился по-турецки {в кресло} с {самшитовыми подлокотниками} и читал]. [Листал страницы {правой рукой}], [а левая сжимала кусок хлеба {с маслом} и сахаром]. [Откусывал задумчиво и читал], [и сахар 

[Древние греки придумали {Олимпийские игры}], [пока вели одну из своих {нескончаемых войн}]. [{Основных причин} было две]: [во-первых], [{во время} баталий солдатам и офицерам некогда было заниматься спортом], [а ведь эллины] ([так называли себя древние греки]) [стремились тренироваться {всё время}], [не {занятое упражнениями} {в философии}]; [во-вторых], [воинам хотелось поскорее вернуться домой], [а отпуск {на войне} не предоставлялся]. [Было ясно], [что войска нуждались {в перемирии} и что {единственной возможностью} его объявить могли стать {Олимпийские игры}]: [ведь {непременное условие} Олимпиады] – [прекращение войны].

[Сначала эллины хотели проводить {Олимпийские игры} ежегодно], [но впоследствии поняли], [что {частые перерывы} {в боевых} действиях бесконечно удлиняют войны], [поэтому {Олимпийские игры} стали объявлять только раз в четыре года]. [{Зимних игр} в {те времена}], [конечно же], [не было], [потому что {в Элладе} не было ни {ледовых арен}], [ни {горнолыжных трасс}].


[В шесть утра], [тщательно одетый и причесанный], [он уже стоял у {пришкольной колокольни} с карманными часами {в руках}]. [Дождавшись], [когда обе стрелки сольются в {единую линию}] ([часовая на шести], [минутная на двенадцати]), [что есть силы дергал {за веревку}] – [и {бронзовый колокол} гулко отзывался]. [За многие годы упражнений Бах достиг в этом деле {такого мастерства}], [что звук удара раздавался ровно в {тот момент}], [когда {минутная стрелка} касалась {циферблатного зенита}], [и ни секундой позже]. [Мгновение спустя каждый {в деревне} поворачивался {на звук} и шептал {короткую молитву}]. [Наступал {новый день}].

[{За годы} учительства], [каждый из которых напоминал предыдущий и ничем особенным не выделялся], [Якоб Иванович настолько привык произносить одни и те же слова и зачитывать одни и те же задачки], [что научился при этом мысленно раздваиваться внутри {своего тела}]: [язык его бормотал текст очередного {грамматического правила}], [рука зажатой в ней линейкой вяло шлеп

In [69]:
for word in (mystem.lemmatize(texts[0])):
    print(word, end=' is ')
    print(getPos(word))

профессорский is ADJF
  is None
дача is NOUN
  is None
на is PREP
  is None
берег is NOUN
  is None
финский is ADJF
  is None
залив is NOUN
.  is None
в is PREP
  is None
отсутствие is NOUN
  is None
хозяин is NOUN
,  is None
друг is NOUN
  is None
мой is ADJF
  is None
отец is NOUN
,  is None
наш is ADJF
  is None
семья is NOUN
  is None
позволяться is INFN
  is None
там is ADVB
  is None
жить is INFN
.  is None
даже is PRCL
  is None
спустя is PREP
  is None
десятилетие is NOUN
  is None
помнить is INFN
,  is None
как is CONJ
  is None
после is PREP
  is None
утомительный is ADJF
  is None
дорога is NOUN
  is None
из is PREP
  is None
город is NOUN
  is None
я is NPRO
  is None
обволакивать is INFN
  is None
прохлада is NOUN
  is None
деревянный is ADJF
  is None
дома is NOUN
,  is None
как is CONJ
  is None
собирать is INFN
  is None
растрясаться is INFN
,  is None
распадаться is INFN
  is None
в is PREP
  is None
экипаж is NOUN
  is None
тело is NOUN
.  is None
этот is ADJF
  is No

музыка is NOUN
  is None
и is CONJ
  is None
приниматься is INFN
  is None
дирижер is NOUN
  is None
в is PREP
  is None
целое is NOUN
  is None
благосклонно is ADVB
.  is None
окружать is INFN
  is None
музыкант is NOUN
,  is None
дирижер is NOUN
  is None
быть is INFN
  is None
в is PREP
  is None
тот is ADJF
  is None
же is PRCL
  is None
время is NOUN
  is None
одинокий is ADJF
:  is None
быть is INFN
  is None
в is PREP
  is None
этот is ADJF
  is None
профессия is NOUN
  is None
непостижимый is ADJF
  is None
трагизм is NOUN
.  is None
он is NPRO
,  is None
быть is INFN
  is None
мочь is NOUN
,  is None
выражать is INFN
  is None
не is PRCL
  is None
так is CONJ
  is None
ярко is ADVB
,  is None
как is CONJ
  is None
у is PREP
  is None
брандмейстер is NOUN
,  is None
поскольку is CONJ
  is None
не is PRCL
  is None
связанный is ADJF
  is None
ни is PRCL
  is None
с is PREP
  is None
огонь is NOUN
,  is None
ни is PRCL
  is None
с is PREP
  is None
внешний is ADJF
  is None
обсто

In [15]:
mystem.analyze(texts[0])

[{'analysis': [{'gr': 'A=им,ед,полн,жен', 'lex': 'профессорский'}],
  'text': 'Профессорская'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,жен,неод=им,ед', 'lex': 'дача'}], 'text': 'дача'},
 {'text': ' '},
 {'analysis': [{'gr': 'PR=', 'lex': 'на'}], 'text': 'на'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=(дат,ед|местн,ед)', 'lex': 'берег'}],
  'text': 'берегу'},
 {'text': ' '},
 {'analysis': [{'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)',
    'lex': 'финский'}],
  'text': 'Финского'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=род,ед', 'lex': 'залив'}], 'text': 'залива'},
 {'text': '. '},
 {'analysis': [{'gr': 'PR=', 'lex': 'в'}], 'text': 'В'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,сред,неод=(вин,ед|им,ед)', 'lex': 'отсутствие'}],
  'text': 'отсутствие'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,од=(вин,ед|род,ед)', 'lex': 'хозяин'}],
  'text': 'хозяина'},
 {'text': ', '},
 {'analysis': [{'gr': 'S,муж,од=(вин,ед|род,ед)', 'lex': 'друг'}],
  'text

In [53]:
x = pymorphy.parse('1917')[0]

In [54]:
x

Parse(word='1917', tag=OpencorporaTag('NUMB,intg'), normal_form='1917', score=1.0, methods_stack=((<NumberAnalyzer>, '1917'),))